In [1]:
# Imports
import soundfile as sf
import pya
import numpy as np
from src import inference
from audio_utils import select_preview_snippet

In [2]:
# Pya Server 
auto_backend = pya.determine_backend()
s = pya.Aserver(backend=auto_backend)
pya.Aserver.default = s 
s.boot()

AServer: sr: 44100, blocksize: 512,
         Stream Active: True, Device: MacBook Air-Lautsprecher, Index: 1

In [19]:
# Model to Use, "musdb" for MUSDB-trained, "exploited" for exploited and "unexploited" for unexploited
model = "exploited"

# Path to Vocal, Piano and Guitar Soundfiles (needs to be in sync)
path_to_vocal_recording = "../../vocal-bleed-removal/tracks/thomas/night/tracks/Voice.wav"
path_to_piano_recording = "../../vocal-bleed-removal/tracks/thomas/night/tracks/Piano.wav"
path_to_guitar_recording = "../../vocal-bleed-removal/tracks/thomas/night/tracks/Guitar.wav"

# Mixing parameter (calculates the mix for the clean sources: clean_mix = alpha * piano + (1-alpha)*guitar
alpha = 0.5

# sample_rate 
sample_rate = 44100

# Post Processing 
use_noise_gate = False 
threshold = -40

# Preview: Select start and end seconds to listen to preview: Set Both to None to make full prediction
start = None 
end = None

In [20]:
# Read Audio Sources and convert to pya signal
vocal = pya.Asig(sf.read(path_to_vocal_recording)[0], sample_rate)
piano = pya.Asig(sf.read(path_to_piano_recording)[0], sample_rate).mono()
guitar = pya.Asig(sf.read(path_to_guitar_recording)[0], sample_rate)

# Select Previews of Songs
vocal = select_preview_snippet(vocal, start, end)
piano = select_preview_snippet(piano, start, end)
guitar = select_preview_snippet(guitar, start, end)

# Create mixture out of clean sources
clean_sources = np.add(piano * alpha, guitar * (1 - alpha))

In [21]:
# Make Predictions with given parameters and save into "/predictions" folder. 
prediction = inference.run_and_save_prediction(model_name=model, vocals=vocal, clean_sources=clean_sources, use_noise_gate=use_noise_gate, threshold=threshold, song_name="song_name")

/Users/wieland/PycharmProjects/thesis-notebook/venv/lib/python3.10/site-packages/keras/src/layers/core/lambda_layer.py:327: UserWarning: wave_u_net is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


1/1 [==============================] - 0s 46ms/step
Saved Prediction in Predictions Folder


In [18]:
prediction.play()

Asig(''): 1 x 441000 @ 44100Hz = 10.000s cn=['0']